# Gestores de contexto y la declaración "with"

La sentencia with en Python es considerada por algunos como una característica oscura. Pero cuando echas un vistazo detrás de las escenas, verás que no hay ninguna magia involucrada, y en realidad es una característica muy útil que puede ayudarte a escribir un código Python más limpio y legible.

Entonces, ¿para qué sirve la sentencia with? Ayuda a simplificar algunos patrones de gestión de recursos comunes al abstraer su funcionalidad y permitir que sean factorizados y reutilizados.

Una buena forma de ver cómo se utiliza esta función de forma efectiva es observando ejemplos de la biblioteca estándar de Python. La función incorporada open() nos proporciona un excelente caso de uso:

In [1]:
with open('hello.txt', 'w') as f: 
    f.write('hello, world!')

La apertura de archivos mediante la sentencia with se recomienda generalmente porque asegura que los descriptores de archivo abiertos se cierren automáticamente después de que la ejecución del programa abandone el contexto de la sentencia with. 

En definitiva, el ejemplo de código anterior se traduce en algo así:

In [2]:
f = open('hello.txt', 'w') 

try:
    f.write('hello, world') 
finally:
    f.close()

Ya puedes ver que esto es bastante más explícito. 

hay que tener en cuenta que la declaración try...finally es significativa. No sería suficiente con escribir algo como esto:

In [3]:
f = open('hello.txt', 'w') 
f.write('hello, world') 
f.close()

Esta implementación no garantizará que el archivo se cierre si hay una excepción durante la llamada a f.write(), y por lo tanto nuestro programa podría filtrar un descriptor de archivo. Por eso la sentencia with es tan útil. Hace que la adquisición y liberación de recursos sea muy fácil.

## Usar with con tus propios objetos

Puedes proporcionar la misma funcionalidad en tus propias clases y funciones implementando los llamados gestores de contexto.

¿Qué es un gestor de contexto? 

Es un simple "protocolo" (o interfaz) que tu objeto debe seguir para soportar la sentencia with. 

Básicamente, todo lo que necesitas hacer es añadir los métodos __enter__ y __exit__ a un objeto si quieres que funcione como un gestor de contexto. Python llamará a estos dos métodos en los momentos adecuados del ciclo de gestión de recursos.

Veamos cómo sería esto en términos prácticos. He aquí cómo podría ser una implementación sencilla del gestor de contexto open():

class ManagedFile:
    
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        self.file = open(self.name, 'w') 
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb): 
        if self.file:
            self.file.close()

Nuestra clase ManagedFile sigue el protocolo del gestor de contexto y ahora soporta la sentencia with, al igual que el ejemplo original de open():

In [5]:
with ManagedFile('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

Python llama a __enter__ cuando la ejecución entra en el contexto de la sentencia with y es el momento de adquirir el recurso. Cuando la ejecución vuelve a salir del contexto, Python llama a __exit__ para liberar el recurso.

Escribir un gestor de contexto basado en una clase no es la única forma de soportar la sentencia with en Python. 

El módulo de utilidad contextlib7 en la biblioteca estándar proporciona algunas abstracciones más construidas sobre el protocolo básico del gestor de contexto. Esto puede hacer tu vida un poco más fácil si tus casos de uso coinciden con lo que ofrece contextlib.


In [7]:
from contextlib import contextmanager
@contextmanager
def managed_file(name): 
    try:
        f = open(name, 'w')
        yield f 
    finally:
        f.close()

In [8]:
with managed_file('hello.txt') as f:
    f.write('hello, world!')
    f.write('bye now')

En este caso, managed_file() es un generador que primero adquiere el recurso. Después, suspende temporalmente su propia ejecución y cede el recurso para que pueda ser utilizado por la persona que llama. Cuando la persona que llama abandona el contexto, el generador continúa ejecutándose para que cualquier paso de limpieza restante pueda ocurrir y el recurso pueda ser liberado de nuevo al sistema.

La implementación basada en la clase y la basada en el generador son básicamente equivalentes. Puedes preferir una sobre la otra, dependiendo de qué enfoque encuentres más legible.

Una desventaja de la implementación basada en @contextmanager puede ser que requiere cierta comprensión de los conceptos avanzados de Python como los decoradores y generadores.

Una vez más, la elección de la implementación correcta se reduce a lo que tú y tu equipo os sintáis cómodos utilizando y a lo que encontréis más legible.

## Escribir APIs bonitas con gestores de contexto

Los gestores de contexto son bastante flexibles, y si utilizas el estado con de forma creativa, puedes definir APIs convenientes para tus módulos y clases.

Por ejemplo, ¿qué pasaría si el "recurso" que quisiéramos gestionar fueran los niveles de sangría del texto en algún tipo de programa generador de informes? ¿Y si pudiéramos escribir un código como este para hacerlo?

In [10]:
class Indenter:
    
    def __init__(self):
        self.level = 0
        
    def __enter__(self): 
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
        
    def print(self, text):
        print(' ' * self.level + text)

In [11]:
with Indenter() as indent:
    indent.print('hi!') 
    with indent:
        indent.print('hello') 
        with indent:
            indent.print('bonjour')
    indent.print('hey')

 hi!
  hello
   bonjour
 hey


## Claves

* La sentencia with simplifica la gestión de excepciones encapsulando los usos estándar de las sentencias try/finally en los llamados gestores de contexto.

* Lo más habitual es que se utilice para gestionar la adquisición y liberación segura de recursos del sistema. Los recursos son adquiridos por la sentencia with y liberados automáticamente cuando la ejecución deja el contexto with.

* Usar with de forma efectiva puede ayudarte a evitar fugas de recursos y hacer que tu código sea más fácil de leer.